In [1]:
import pandas as pd
from sympy import false
from sympy.physics.units import farad

In [2]:
# --- 配置部分 ---
file_path = './附件3_处理.xlsx'

sheet_name = 'Sheet1' # 例如 'Sheet1', 'Sheet2' 等

# 输出文件路径
output_file_path = '附件3_处理_final.xlsx'

# --- 配置部分结束 ---

In [3]:
df_orginal = pd.read_excel(file_path, sheet_name=sheet_name, header=0)

print(f"成功读取文件：{file_path}，工作表：{sheet_name}")
print("原始数据（部分）：")
print(df_orginal.head())

成功读取文件：./附件3_处理.xlsx，工作表：Sheet1
原始数据（部分）：
   餐次   序号 食物名称 主要成分     食物编码  可食部\n（克/份）  价格\n（元/份） 是否\n可半份
0  早餐  1.0   牛奶   牛奶  101101x         200        1.5       否
1  早餐  2.0   酸奶   酸奶   103001         125        1.5       否
2  早餐  3.0   豆浆   黄豆    31101          10        1.5       否
3  早餐  4.0  大米粥   稻米  012001x          15        0.5       否
4  早餐  5.0  小米粥   小米    15101          15        0.5       否


## 填充

In [4]:
# --- 自动识别需要填充的列 ---
# 找到所有包含 NaN (空白) 值的列
columns_with_blanks = df_orginal.columns[df_orginal.isnull().any()].tolist()

if not columns_with_blanks:
    print("\n数据中没有发现任何空白单元格，无需填充。")

else:
    print(f"\n识别到需要填充的列：{columns_with_blanks}")

    # 对自动识别到的列进行向下填充 (ffill = forward fill)
    for col in columns_with_blanks:
         df_orginal[col] = df_orginal[col].ffill()
         # 打印，但为了简洁，只在处理完成后打印一次列表
         # print(f"已对列 '{col}' 进行向下填充。")


    print(f"已对识别到的 {len(columns_with_blanks)} 列进行向下填充。")
    print("\n填充后数据（部分）：")
    print(df_orginal.head())



识别到需要填充的列：['序号', '食物名称', '价格\n（元/份）', '是否\n可半份']
已对识别到的 4 列进行向下填充。

填充后数据（部分）：
   餐次   序号 食物名称 主要成分     食物编码  可食部\n（克/份）  价格\n（元/份） 是否\n可半份
0  早餐  1.0   牛奶   牛奶  101101x         200        1.5       否
1  早餐  2.0   酸奶   酸奶   103001         125        1.5       否
2  早餐  3.0   豆浆   黄豆    31101          10        1.5       否
3  早餐  4.0  大米粥   稻米  012001x          15        0.5       否
4  早餐  5.0  小米粥   小米    15101          15        0.5       否


## 替换是否为1,0

In [5]:
replacement_map = {
    '是': 1,
    '否':0,
}

column_to_replace = '是否\n可半份'
df_orginal[column_to_replace] = df_orginal[column_to_replace].replace(replacement_map).infer_objects(copy=False)
df_orginal[column_to_replace]

C:\Users\Sophomores\AppData\Local\Temp\ipykernel_3432\842274652.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_orginal[column_to_replace] = df_orginal[column_to_replace].replace(replacement_map).infer_objects(copy=False)


0      0
1      0
2      0
3      0
4      0
      ..
377    0
378    0
379    0
380    0
381    0
Name: 是否\n可半份, Length: 382, dtype: int64

## 添加数据并合并

"碳水化合物 (g/100g)",
"蛋白质 (g/100g)",
"脂肪 (g/100g)",
"钙 (mg/100g)",
"铁 (mg/100g)",
"锌 (mg/100g)",
"维生素A (μg/100g)",
"维生素B1 (mg/100g)",
"维生素B2 (mg/100g)",
"维生素C (mg/100g)",
"异亮氨酸 (g/100g)",
"亮氨酸 (g/100g)",
"赖氨酸 (g/100g)",
"含硫氨基酸 (g/100g)",
"芳香族氨基酸 (g/100g)",
"苏氨酸 (g/100g)",
"色氨酸 (g/100g)",
"缬氨酸 (g/100g)"


In [6]:
merge_reference_file_path = r'2024电工杯B题 所有指标(完整表格) .xlsx'
df_merge_reference = pd.read_excel(merge_reference_file_path)
# 打印出来，查看具体的元组名称
df_merge_reference.columns


Index(['主要成分', '食物编码', '水分', '能量(kcal)', '能量(kJ)', '蛋白质', '脂肪', '碳水化合物',
       '不溶性膳食纤维', '胆固醇', '灰分', '总维生素A', '胡萝卜素', '视黄醇', '硫胺素', '核黄素', '烟酸',
       '维生素C', '钙', '磷', '钾', '钠', '镁', '铁', '锌', '硒', '铜', '锰', '蛋白质.1',
       '异亮氨酸', '亮氨酸', '赖氨酸', '含硫氨基酸(SAA)_Total', '含硫氨基酸(SAA)_蛋氨酸',
       '含硫氨基酸(SAA)_胱氨酸', '芳香族氨基酸(AAA)_Total', '芳香族氨基酸(AAA)_苯丙氨酸',
       '芳香族氨基酸(AAA)_酪氨酸', '苏氨酸', '色氨酸', '缬氨酸', '精氨酸', '组氨酸', '丙氨酸', '天冬氨酸',
       '谷氨酸', '甘氨酸', '脯氨酸', '丝氨酸'],
      dtype='object')

In [7]:
# --- 合并的列名 ---
df_merge_reference = df_merge_reference.drop(columns='主要成分')
merge_column = '食物编码' # 用于合并的列名，确保在两表中都存在
print(f"\n正在执行合并操作，使用列 '{merge_column}' 进行左合并...")
# how='left' 表示保留 df_original 中的所有行，并将 df_amino 中匹配的数据添加到对应行

df_merged = pd.merge(
    df_orginal,      # 左表 (主表)
    df_merge_reference,         # 右表 (已简化列名的表)
    on=merge_column,  # 指定合并的列，现在两边都是单层字符串名称
    how='left'        # 合并方式，这里使用左合并
)
print("合并操作完成。")

print("\n合并后的数据（部分）：")
print(df_merged.head())


正在执行合并操作，使用列 '食物编码' 进行左合并...
合并操作完成。

合并后的数据（部分）：
   餐次   序号 食物名称 主要成分     食物编码  可食部\n（克/份）  价格\n（元/份）  是否\n可半份    水分  能量(kcal)  \
0  早餐  1.0   牛奶   牛奶  101101x         200        1.5        0  87.6        65   
1  早餐  2.0   酸奶   酸奶   103001         125        1.5        0  85.5        70   
2  早餐  3.0   豆浆   黄豆    31101          10        1.5        0  10.2       390   
3  早餐  4.0  大米粥   稻米  012001x          15        0.5        0  13.3       346   
4  早餐  5.0  小米粥   小米    15101          15        0.5        0  11.6       361   

   ...  色氨酸   缬氨酸   精氨酸  组氨酸   丙氨酸  天冬氨酸   谷氨酸   甘氨酸   脯氨酸   丝氨酸  
0  ...   54   178    94   78    94   229   639    54   289   166  
1  ...   48   155    95   74    94   217   590    53   238   150  
2  ...  455  1726  2840  968  1542  3997  6258  1600  1863  1846  
3  ...  124   426   570  161   420   650  1250   305   324   360  
4  ...  178   483   315  168   803   682  1871   245   658   408  

[5 rows x 55 columns]


In [8]:
df_merged.to_excel(output_file_path, index=False)
print("数据保存完成。")

数据保存完成。
